# Homework 5: Time Series & Gridded Climate Data
## F&W ECOL 458: Environmental Data Science
### Covers Lectures 10 & 11

**Due:** *[see Canvas]*

**Instructions:**
- Complete all code cells marked with `# YOUR CODE HERE`
- Answer all written-response questions in the provided markdown cells
- Run all cells to make sure your code works before submitting
- Submit the completed `.ipynb` file to Canvas

---

## Setup

In [ ]:
# Run this cell first — installs and imports
!pip install xarray netCDF4 statsmodels cartopy -q

In [ ]:
# Mount Google Drive (the NetCDF file is stored there)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from statsmodels.tsa.seasonal import seasonal_decompose

import warnings
warnings.filterwarnings('ignore')

---
## Problem 1: Time Series Analysis of Simulated Weather Data (50 pts)

In this problem you will practice the core time-series skills from Lecture 10: resampling, rolling averages, and seasonal decomposition. We'll use a synthetic daily weather dataset for Madison, WI that has realistic seasonal patterns.

Run the cell below to generate the data — **do not modify it.**

In [ ]:
# === DO NOT MODIFY THIS CELL — it creates the dataset ===
np.random.seed(458)

dates = pd.date_range('2018-01-01', '2024-12-31', freq='D')
n = len(dates)
doy = dates.dayofyear

# Seasonal cycle + slight warming trend + noise
seasonal = 16 * np.sin(2 * np.pi * (doy - 80) / 365)
trend = np.linspace(0, 1.5, n)
noise = np.random.normal(0, 4, n)
temperature = 8 + seasonal + trend + noise  # °C

# Precipitation (exponential with seasonal modulation)
precip_rate = 2.5 + 1.0 * np.sin(2 * np.pi * (doy - 120) / 365)
precipitation = np.random.exponential(precip_rate)  # mm

weather = pd.DataFrame({
    'date': dates,
    'temperature_c': temperature,
    'precipitation_mm': precipitation
})

# Set date as index (DatetimeIndex)
weather = weather.set_index('date')

print(f"Dataset: {len(weather)} days, {weather.index[0].date()} to {weather.index[-1].date()}")
weather.head()

### Part A — Resampling (15 pts)

**A1 (5 pts).** Resample the daily temperature data to **monthly means**. Store the result in a variable called `monthly_temp`. Print the first 12 values.

In [ ]:
# YOUR CODE HERE


**A2 (5 pts).** Resample the daily precipitation data to **annual totals** (sum). Store the result in a variable called `annual_precip`. Print all values.

In [ ]:
# YOUR CODE HERE


**A3 (5 pts).** Make a bar plot of `annual_precip` with year on the x-axis and total precipitation (mm) on the y-axis. Add axis labels and a title.

In [ ]:
# YOUR CODE HERE


### Part B — Rolling Averages (15 pts)

**B1 (5 pts).** Calculate a **30-day rolling mean** of temperature. Store it in a new column called `temp_30day_ma`.

In [ ]:
# YOUR CODE HERE


**B2 (10 pts).** Create a figure that shows **one year of data (2023)** with:
- The raw daily temperature as a thin, semi-transparent line
- The 30-day rolling mean as a thicker line on top
- A legend, axis labels, and a title

*Hint:* Use `.loc['2023']` to select the year.

In [ ]:
# YOUR CODE HERE


### Part C — Seasonal Decomposition (20 pts)

**C1 (5 pts).** Use `seasonal_decompose()` from statsmodels to decompose `monthly_temp` (the monthly means you created in Part A). Use `period=12` for an annual cycle. Store the result in a variable called `decomp`.

In [ ]:
# YOUR CODE HERE


**C2 (5 pts).** Plot the four decomposition panels (observed, trend, seasonal, residual) using `decomp.plot()`. Set the figure size to (14, 10).

In [ ]:
# YOUR CODE HERE


**C3 (10 pts).** Answer the following questions in 1–2 sentences each.

1. Looking at the **trend** component: does temperature appear to be increasing, decreasing, or staying the same over the 7-year record?
2. Looking at the **seasonal** component: approximately what is the range (warmest minus coldest) of the annual temperature cycle in °C?
3. Why did we use `period=12` instead of `period=365` here?

*Your answers:*

1. 
2. 
3. 


---
## Problem 2: Gridded Climate Data with xarray (50 pts)

In this problem you will work with the real **CRU TS4.09 temperature dataset** (CONUS subset) from Lecture 11. You'll practice selecting data by coordinates and time, computing climate anomalies, and drawing a map.

Run the cell below to load the data — **do not modify it.**

In [ ]:
# === DO NOT MODIFY THIS CELL — it loads the CRU dataset ===
cru_path = '/content/drive/MyDrive/cru_ts4.09.1901.2024.tmp.conus.nc'
ds = xr.open_dataset(cru_path)

print(f"Loaded CRU TS4.09 CONUS temperature dataset")
print(f"  Time: {pd.to_datetime(ds.time.values[0]).date()} to {pd.to_datetime(ds.time.values[-1]).date()}")
print(f"  Lat:  {float(ds.lat.min()):.2f}°N to {float(ds.lat.max()):.2f}°N")
print(f"  Lon:  {float(ds.lon.min()):.2f}° to {float(ds.lon.max()):.2f}°")
print(f"  Shape (time, lat, lon): {ds['tmp'].shape}")
ds

### Part A — Selecting Data (15 pts)

**A1 (5 pts).** Extract the temperature time series for a grid cell near **Denver, CO** (latitude ≈ 39.7°N, longitude ≈ −105.0°). Use `method='nearest'` in `.sel()`. Store the result in a variable called `denver_temp`. Print the shape and the coordinates of the nearest grid point.

In [ ]:
# YOUR CODE HERE


**A2 (5 pts).** Select all temperature data for the month of **January 2010**. Store it in a variable called `jan_2010`. Print its shape.

In [ ]:
# YOUR CODE HERE


**A3 (5 pts).** Extract a regional subset covering the **Southeast US**: latitude 25°N to 37°N, longitude −92° to −75°. Store it in a variable called `southeast`. Print the number of latitude and longitude grid cells.

In [ ]:
# YOUR CODE HERE


### Part B — Climate Anomalies (20 pts)

**B1 (5 pts).** Calculate the **1991–2020 monthly climatology** (i.e., the average January, average February, … average December over that 30-year baseline). Store the result in a variable called `climatology`.

*Hint:* First select the baseline period with `.sel(time=slice(...))`, then use `.groupby('time.month').mean(dim='time')`.

In [ ]:
# YOUR CODE HERE


**B2 (5 pts).** Compute the temperature **anomaly** for the full record by subtracting the climatology from each month. Store the result in a variable called `anomaly`.

*Hint:* `anomaly = ds['tmp'].groupby('time.month') - climatology`

In [ ]:
# YOUR CODE HERE


**B3 (10 pts).** Using the Denver grid cell from Part A, plot the **monthly anomaly time series** at that location. Your plot should include:
- The anomaly values as a line plot
- A horizontal dashed line at y = 0
- Axis labels and a title that mentions the baseline period

In [ ]:
# YOUR CODE HERE


### Part C — Drawing a Map (15 pts)

**C1 (15 pts).** Create a map of the **Summer 2012 (June–August) CONUS temperature anomaly** using cartopy. Summer 2012 was one of the hottest summers on record for the US, so this is a great case to visualize.

Your map should include:
- A **diverging colormap** centered at 0 (e.g., `cmap='RdBu_r', center=0`)
- **State boundaries** (`cfeature.STATES`)
- **Coastlines**
- A **colorbar** with a label
- A **title**

You may use either Plate Carrée or Lambert Conformal Conic projection.

*Hint:* First compute the summer 2012 mean anomaly with:
```python
summer_2012 = anomaly.sel(time=slice('2012-06', '2012-08')).mean(dim='time')
```
Then follow the cartopy recipe from Lecture 11.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# YOUR CODE HERE
